David Shaffer
Anika Rahman

This ETL project will load datasets found on Kaggle on Baseball statistics from the [History of Baseball](https://www.kaggle.com/seanlahman/the-history-of-baseball)

We use three CSVs found on this website: batting data, player data, and salary data. We extract data starting from the years 1998 and the seasons where the batters had more than 100 at-bats. After cleaning the data, we load it into SQL (relational) to join a final table that contains salary data for the most active baseball batters in the past 20 years.

In [1]:
#Import dependencies

import pymysql
import pandas as pd
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()

In [37]:
#Import datasets

batting = 'batting.csv'
salary = 'salary.csv'
player = 'player.csv'
batting_df = pd.read_csv(batting)
salary_df = pd.read_csv(salary)
player_df = pd.read_csv(player)
display(batting_df.head())
display(salary_df.head())
display(player_df.head())

,player_id,year,stint,team_id,league_id,g,ab,r,h,double,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,g_idp
0,abercda01,1871,1,TRO,NaN,1,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118.0,30.0,32.0,6.0,...,13.0,8.0,1.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137.0,28.0,40.0,4.0,...,19.0,3.0,1.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133.0,28.0,44.0,10.0,...,27.0,1.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120.0,29.0,39.0,11.0,...,16.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN


,year,team_id,league_id,player_id,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [38]:
#Add Batting Average column
batting_data = batting_df
batting_data['average'] = batting_data['h'] / batting_data['ab']

#Filter for 1998 and more than 100 at bats to filter out pitchers and batters that didn't play much
batting_data = batting_data.loc[(batting_data['year'] >= 1998) & (batting_data['ab'] >= 100) ]

#Clean data
batting_data = batting_data[['player_id', 'hr', 'rbi', 'average', 'year']]
batting_data = batting_data.fillna(value = 0)
batting_data['hr'] = batting_data['hr'].astype(int)
batting_data['rbi'] = batting_data['rbi'].astype(int)
batting_data = batting_data.rename(columns = {'hr': 'Home_Runs', 'rbi': 'Runs_Batted_In', 'average': 'Batting_Average', 'year': 'Year'})
batting_data.set_index('player_id', inplace = True)

#Clean data
player_data = player_df
player_data = player_data[['player_id', 'name_first', 'name_last']]
player_data = player_data.rename(columns = {'name_first': 'First_Name', 'name_last':'Last_Name'} )
player_data.set_index('player_id', inplace = True)

#Clean data
salary_data = salary_df
salary_data = salary_data[['player_id', 'salary', 'year']]
salary_data = salary_data.rename(columns = {'salary':'Salary', 'year': 'Year'})
salary_data.set_index('player_id', inplace = True)

display(batting_data.head())
display(salary_data.head())
display(player_data.head())

,Home_Runs,Runs_Batted_In,Batting_Average,Year
player_id,,,,
abbotje01,12,41,0.278689,1998
abbotku01,2,9,0.268293,1998
abreubo01,17,74,0.311871,1998
alexama02,5,25,0.227273,1998
alfoned01,17,78,0.278276,1998


,Salary,Year
player_id,,
barkele01,870000,1985
bedrost01,550000,1985
benedbr01,545000,1985
campri01,633333,1985
ceronri01,625000,1985


,First_Name,Last_Name
player_id,,
aardsda01,David,Aardsma
aaronha01,Hank,Aaron
aaronto01,Tommie,Aaron
aasedo01,Don,Aase
abadan01,Andy,Abad


In [5]:
#Create engine for sqlite

engine = create_engine(f'sqlite:///baseball.db')

In [6]:
engine.execute("""
DROP TABLE IF EXISTS salary
""")

In [7]:
engine.execute("""
DROP TABLE IF EXISTS player
""")

In [8]:
engine.execute("""
DROP TABLE IF EXISTS batting
""")

In [9]:
#Create Player table

engine.execute("""
CREATE TABLE player (
  player_id TEXT PRIMARY KEY,
  First_Name TEXT,
  Last_Name TEXT
)
""")

In [10]:
#Create Batting table

engine.execute("""
CREATE TABLE batting (
  player_id TEXT PRIMARY KEY,
  Home_Runs INT,
  Runs_Batted_In INT,
  Batting_Average DECIMAL(3,3),
  Year INT
)
""")

In [11]:
#Create Salary table

engine.execute("""
CREATE TABLE salary (
  player_id TEXT PRIMARY KEY,
  Salary INT,
  Year INT
)
""")

In [12]:
#Load the Dataframes into sqlite

batting_data.to_sql(name='batting', con=engine,
                            chunksize=100, 
                            if_exists='replace', index=True)

salary_data.to_sql(name='salary', con=engine,
                            chunksize=100, 
                            if_exists='replace', index=True)

player_data.to_sql(name='player', con=engine,
                            chunksize=100, 
                            if_exists='replace', index=True)

In [42]:
#Read tables from SQL

display(pd.read_sql_query(("""
SELECT *
FROM batting
LIMIT 10
"""), con=engine))

display(pd.read_sql_query(("""
SELECT *
FROM salary
LIMIT 10
"""), con=engine))

display(pd.read_sql_query(("""
SELECT *
FROM player
LIMIT 10
"""), con=engine))

,player_id,Home_Runs,Runs_Batted_In,Batting_Average,Year
0,abbotje01,12,41,0.278689,1998
1,abbotku01,2,9,0.268293,1998
2,abreubo01,17,74,0.311871,1998
3,alexama02,5,25,0.227273,1998
4,alfoned01,17,78,0.278276,1998
5,alicelu01,6,33,0.274131,1998
6,allenje01,3,24,0.309013,1998
7,alomaro01,14,56,0.282313,1998
8,alomasa02,6,44,0.234719,1998
9,aloumo01,38,124,0.311644,1998


,player_id,Salary,Year
0,barkele01,870000,1985
1,bedrost01,550000,1985
2,benedbr01,545000,1985
3,campri01,633333,1985
4,ceronri01,625000,1985
5,chambch01,800000,1985
6,dedmoje01,150000,1985
7,forstte01,483333,1985
8,garbege01,772000,1985
9,harpete01,250000,1985


,player_id,First_Name,Last_Name
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad
5,abadfe01,Fernando,Abad
6,abadijo01,John,Abadie
7,abbated01,Ed,Abbaticchio
8,abbeybe01,Bert,Abbey
9,abbeych01,Charlie,Abbey


In [36]:
#Join the 3 SQL tables together using a SQL join and read the data back

combined = pd.read_sql_query(("""
SELECT b.*,s.Salary, s.Year, p.First_Name, p.Last_Name
FROM batting as b
INNER JOIN salary as s
ON b.player_id = s.player_id AND b.year = s.year
INNER JOIN player as p
on p.player_id = b.player_id;
"""), con=engine)

combined

,player_id,Home_Runs,Runs_Batted_In,Batting_Average,Year,Salary,Year,First_Name,Last_Name
0,abbotje01,12,41,0.278689,1998,175000,1998,Jeff,Abbott
1,abbotku01,2,9,0.268293,1998,1000000,1998,Kurt,Abbott
2,abreubo01,17,74,0.311871,1998,180000,1998,Bobby,Abreu
3,alexama02,5,25,0.227273,1998,425000,1998,Manny,Alexander
4,alfoned01,17,78,0.278276,1998,1750000,1998,Edgardo,Alfonzo
5,alicelu01,6,33,0.274131,1998,750000,1998,Luis,Alicea
6,allenje01,3,24,0.309013,1998,260000,1998,Jermaine,Allensworth
7,alomaro01,14,56,0.282313,1998,6343771,1998,Roberto,Alomar
8,alomasa02,6,44,0.234719,1998,2725000,1998,Sandy,Alomar
9,aloumo01,38,124,0.311644,1998,5020000,1998,Moises,Alou
